In [451]:
from matplotlib import pyplot as plt 
import numpy as np 
import scipy as sp
import mph

Code here is to Modify the parameters and Output the streamline and electric field automatically.

In [452]:
client=mph.start(cores=3)

In [453]:
model= client.load("reproduce.mph")
print("client names  :{}".format(client.names() ))
print("client models :{}".format(client.models()))

client names  :['reproduce']
client models :[Model('reproduce')]


In [454]:
base_dir="K:\Comsol/reproduce"

## Parameter Modifications

In [455]:
# Load Original Parameter
for (name, value) in model.parameters().items():
    description = model.description(name)
    print(f'{description:20} {name} = {value}')

                     up_radius = 5[um]
                     down_radius = 5[um]
                     anode_radius = 9.5[um]
                     up_distance = 3[mm]
                     down_distance = 3[mm]
                     vertical_distance = 3[mm]
                     up_stagger_distance = 0[mm]
                     down_stagger_distance = 0[mm]
                     drift_length = 40[mm]
                     number = 8
                     up_voltage = 0[V]
                     anode_voltage = 9400[V]
                     down_voltage = 0[V]
                     cathode_voltage = 100[V]
                     drift_ratio = 1
                     cathode_distance = 10[mm]
                     cathode_height = 1[mm]


In [456]:
# Parameter modification: Stagger
anode_radius=9.0
anode_voltage=9400
up_stagger_distance=0
down_stagger_distance=0
down_voltage=0
shield_radius=5

model.parameter("drift_ratio",1)
model.parameter("anode_radius","{}[um]".format(anode_radius))
model.parameter("anode_voltage","{}[V]".format(anode_voltage))
model.parameter("up_stagger_distance","{}[mm]".format(up_stagger_distance))
model.parameter("down_stagger_distance","{}[mm]".format(down_stagger_distance))

model.parameter("down_voltage","{}[V]".format(down_voltage))
model.parameter("cathode_voltage","{}[V]".format(down_voltage+100))
model.parameter("drift_ratio","{}".format(1))

model.parameter("up_radius","{}[um]".format(shield_radius))
model.parameter("down_radius","{}[um]".format(shield_radius))

In [457]:
# Check Parameter
for (name, value) in model.parameters().items():
    description = model.description(name)
    print(f'{description:20} {name} = {value}')

                     up_radius = 5[um]
                     down_radius = 5[um]
                     anode_radius = 9.0[um]
                     up_distance = 3[mm]
                     down_distance = 3[mm]
                     vertical_distance = 3[mm]
                     up_stagger_distance = 0[mm]
                     down_stagger_distance = 0[mm]
                     drift_length = 40[mm]
                     number = 8
                     up_voltage = 0[V]
                     anode_voltage = 9400[V]
                     down_voltage = 0[V]
                     cathode_voltage = 100[V]
                     drift_ratio = 1
                     cathode_distance = 10[mm]
                     cathode_height = 1[mm]


### Check Material

In [458]:
# model basic information
print("model material: {}".format(model.materials()))
print("model physics : {}".format(model.physics()))
print("model studies : {}".format(model.studies()))

model material: ['Copper [solid,residual resistivity ratio of 3000]', 'Xenon [liquid]']
model physics : ['Electrostatics']
model studies : ['Study 1']


## Comsol Simulation 

In [459]:
# Check study list
model.studies()

['Study 1']

In [460]:
# The calculation are done within the Comsol software itself , we usually does not use it .

# Geometry
model.geometries()
model.build()

# Mesh 
model.mesh()

# Solution
model.solve(model.studies()[0])

# Dataset
model.datasets()

['Study 1//Solution 1']

In [461]:
model.exports()

['field', 'streamline']

## Export electric field

In [462]:
# field files : raw field txt file , converted h5 file
model.export("field",r"{}/field.txt".format(base_dir))

In [463]:
with open(r"{}/field.txt".format(base_dir),"r",encoding="utf8") as f:
    i=0
    length_u=""
    electric_x_u=""
    electric_y_u=""
    dimension=2
    while True :
        line=f.readline()
        if not line :
            break 
        if i==3 : 
            line_list= [x.strip() for x in line.split(" ") if x!=""]
            dimension= int(line_list[2])
        elif i==7 :
            line_list= [x.strip() for x in line.split(" ") if x!=""]
            length_u= line_list[3]
        elif i==8 : 
            line_list= [x.strip() for x in line.split(" ") if x!=""]
            electric_x_u = line_list[4]
            electric_y_u = line_list[6]
            info=f.readlines()
            break
        i=i+1

for i in range(len(info)):
    info[i]= info[i].split()
info=np.array(info).astype(float)
info=info.T

if length_u=="µm":
    length_u_n=1e-6
elif length_u=="um":
    length_u_n=1e-6
elif length_u=="nm":
    length_u_n=1e-9
elif length_u=='mm':
    length_u_n=1e-3
elif length_u=='m':
    length_u_n=1
else :
    print("Warning!! Unrecognized unit of length_u")

In [464]:
import h5py as h5 
h5f= h5.File(r"{}/singlecell4.h5".format(base_dir),"w")
h5f.create_dataset("info",data=info)
h5f.close()

## Export Streamline

In [465]:
# export the fieldlines ans save it within a single txt file 
model.export("streamline",r"{}/streamline.txt".format(base_dir))

In [466]:
# convert the field lines file into a h5 file 
with open(r"{}/streamline.txt".format(base_dir),"r") as f: 
    for i in range(8) : 
        f.readline()
    stream = f.readlines()
    
for i in range(len(stream)):
    stream[i]= stream[i].split()
stream=np.array(stream).astype(float)
stream=stream.T  

In [467]:
import h5py as h5 
h5f= h5.File(r"{}\singlecell_stream.h5".format(base_dir),"w")
h5f.create_dataset("stream",data=stream)
h5f.close()

## Export Parameters

In [468]:
# Log all the parameters and export it to a txt file 
parameter=model.parameters()
parameter['length_u']=length_u_n
parameter['electric_x_u']= electric_x_u
parameter['electric_y_u']= electric_y_u
parameter['dimension']=dimension
parameter['stream_number']=int(stream[2].max()+1)

with open(r"{}/parameter.txt".format(base_dir),"w") as f : 
    f.write(str(parameter))

# Clear the client models 

In [469]:
model.save()

In [470]:
client.clear()
print(client.names())

[]
